# Week 8: The Heat Equation and the Crank-Nicholson Method

## Library Imports Go Here

In [12]:
import numpy as np
import matplotlib.pyplot as plt
import time
import scipy.sparse as sparse
import scipy.sparse.linalg as linalg

from matplotlib.animation import FuncAnimation
from IPython.display import HTML

## The Numerical Algorithms for the Heat Equation

<font color = blue>
We want code that will solve (find $\tau(s)$) for the generic heat equation
    $$
    \lambda(s)\tau''(s) + \lambda'(s)\tau'(s) = -h(s)
    $$
where the functions $\lambda(s)$ and $h(s)$ are given, and the boundary conditions for $\tau(s)$ are:
    $$
    \tau(0) = 0 \, , \hspace{.5in} \tau(1) = 1
    $$
In order to do this, we convert the equation into a matrix problem of the form
    $$
    \mathbb{M}\vec{\tau} = -(\vec{h} + \vec{b})
    $$

(where the matrix $\mathbb{M}$ encodes the differential operator on the right-hand-side, the vector $\vec{h}$ encodes the source function, and the vector $\vec{b}$ encodes the boundary conditions).

### Generating the Matrix

<font color = blue>
    
Create a function that will build the matrix $\mathbb{M}$.  Your matrix should take as inputs the size of the matrix (that is, of the spatial grid), and the function $\lambda(s)$, and should use produce a sparse array.  You should be able to start with the function you created in the previous part of the assignment and create a function which uses it.  Use a symmetric numerical derivative to find the function $\lambda'(s)$ with an $\epsilon$ parameter set to be 1/100 of the grid spacing.

### Generating the Source Vector

<font color = blue>
    
Create a function that will build the vector $\vec{h}$.  It should take as inputs the size of the vector and the source function $h(s)$.  (Note that this is not typically sparse, so there's no point in using that technology).

### Generating the Boundary Vector

<font color = blue>
Create a function that generates the vector $\vec{b}$ in which you will encode your boundary conditions.  It should take as inputs the size of the vector and the source function $\lambda(s)$, and use the same mechanism as before to calculate the numerical derivative $\lambda'(s)$.  

```
```

We could work hard at this and generate this in sparse form, but it turns out not to be super-important.  The "sparse" structure is designed to make sure we don't use up memory space in the computer storing a bunch of zeroes: instead we only store those elements of the matrix or vector which are non-zero.  But in this problem the matrix you have just created will have of order $N$ non-zero elements.  If we now store all $N$ elements of the vector $\vec{b}$, even those which aren't zero, we haven't changed the scaling of the number of values we are storing (it is still just of order $N$).

### The Solution Maker

<font color = blue>
We now want to use these pieces to create solutions to our differential equation

#### Solution Function

<font color = blue>
    Write a function that will take as inputs the size of the grid $N$, the source function $h(s)$ and conductivity $\lambda(s)$, and will output the solution to the differential equation in the form of two lists $[s_i]$ and $[\tau_i]$.  Your function should use the function scipy.sparse.linalg.inv to invert the matrix.

#### A Simple Test

<font color = blue>
Test your function with the case $\lambda(s) = 1$ and $h(s) = 0$, using $N = 1000$.  Confirm that the solution matches the expected analytic solution $\tau(s) = s$.

## * Exploring the Heat Equation

<font color = blue>
    Now that we have the basic elements to produce solutions to the heat equation in 1D, we want to experiment with them.

### Linearly Varying Conductivity

<font color = blue>
    
Consider the case where the conductivity of the rod varies linearly along it, in the form
    $$
    \lambda(s) = 1 + \beta s
    $$
    
but there is still no source term.

#### Comparing with the Analytic Solution

<font color = blue>
In this case, it is also possible to find the solution analytically, and it is given by

$$
\tau(s) = \frac{\ln(1 + \beta s)}{\ln(1 + \beta)}
$$
    
Use your algorithm to find a solution for $\beta = 1$, and graph it together with the analytic solution.  **Discuss** the results.

#### Varying $\beta$

<font color = blue>
    
Now create a graph which shows the solutions for $\beta = -0.99, -0.9, 0, 10, 100$.   **Discuss** the results.

### Sinusoidally Varying Conductivity

<font color = blue>
    
Now let's consider the case where the conductivity varies sinusoidally across the rod (so parts of the rod which have high conductivity are interspersed with parts of the rod which have low conductivity).  Let's say
    $$
    \lambda(s) = 1 + A\sin(k s)
    $$
where $A = 0.99$ and $k = 20$.  Create plots of both the conductivity itself and the temperature across the rod.   **Discuss** the results.

### Source Terms

<font color = blue>
    Now consider a rod with constant conductivity $\lambda(s) = 1$, but with a non-trivial source term in the form
    $$
    h(s) = \frac{h_0}{\sigma\sqrt{\pi}}e^{-(s - s_0)^2/\sigma^2}
    $$
    
This Gaussian function is appropriate for modeling sources that are nearly point-like, with $s_0$ giving the location of the source, $h_0$ giving its strength, and $\sigma$ giving the width it is spread out over.  (Choosing $\sigma$ small makes it closer to a point-like source).

#### The Significance of $h_0$

<font color = blue>
Create a single graph that shows solutions with $\sigma = 0.01$ and $s_0 = 0.5$, but with $h_0$ in the set

$$
h_0 \in \{-10, 1, 0, 1, 10\}
$$

 **Discuss** the results.

#### The Significance of $s_0$

<font color = blue>
    Now create a graph that shows solutions where $h_0 = 10$ and $\sigma = 0.01$, but
    
$$
s_0 \in \{0.25, .5, 0.75\}
$$

 **Discuss** the results.

#### The Significance of $\sigma$

<font color = blue>
    Finally, create a graph that shows solutions with $h_0 = 10$ and $s_0 = 0.5$, but with
    
$$
\sigma =\in \{0.5, 0.1, 0.05, 0.01, 0.005\}
$$

 **Discuss** the results.

## Basic Code for the Crank-Nicholson Method

### Generating Finite Diference Matrices

#### A Generic Sparse Matrix

<font color = blue>
Begin by copy-pasting your finite-difference code for generating a sparse coo-matrix $\mathbb{M}$ associated to an operator

$$
\hat{M} = A(x)\frac{\partial^2}{\partial x^2} + B(x)\frac{\partial}{\partial x} + C(x)
$$

given three functions $A(x)$, $B(x)$, and $C(x)$ into the space below.  You will then want to modify it slightly to account for the difference in the spatial grid we are now using as opposed to our old choice.

#### The Hamiltonian Matrix

<font color = blue>
    
Next, use that code to write a function which takes as input the dimensioness potential $V(x)$, and the spatial grid parameters $x_{\infty}$ and `matrix_size` = $N$, and outputs the Hamiltonian coo-type matrix $\mathbb{H}$.

### Constructing the Time-Evolution Code

#### Creating the T-Matrix

<font color = blue>
    
Now, write a  function that takes as inputs the dimensionless potential $V(x)$, the spatial grid parameters $x_{\infty}$ and `matrix_size` = $N$, and the time grid parameters `final_time` = $T$ and `time_count` = $M$, and outputs the matrix $\mathbb{T}$ defined as

$$
\mathbb{T} = \left(\mathbb{I} - \frac{i\Delta_t}{2}\mathbb{H}\right)\left(\mathbb{I} + \frac{i\Delta_t}{2}\mathbb{H}\right)^{-1}
$$

Test your code using $V(x) = 0$, $x_{\infty} = 1$, $N = 4$, $T = 10$, and $M = 1000$, and confirm that the matrix generated is unitary (meaning its conjugate transpose is also its inverse.)

#### The Time Evolution Code

<font color = blue>
Now we will put the pieces together and generate the essential piece of code.  It should take as inputs the initial wavefunction $\Psi(x, 0) = \psi(x)$, the potential $V(x)$, the spatial grid parameters $x_{\infty}$ and $N$, and the time grid parameters $T$ and $M$.  It should output a "list of lists", structured as

$$
[\vec{\Psi}^{0}, \vec{\Psi}^{1}, \vec{\Psi}^{2}, \dots ] = [[\Psi^0_1, \Psi^0_2, \dots, \Psi^0_N], [\Psi^1_1, \Psi^1_2, \dots, \Psi^1_N], \dots, [\Psi^M_1, \Psi^M_2, \dots, \Psi^M_N]]
$$

Note that the first thing your code will need to do is convert the initial function $\psi(x)$ into the vector $\vec{\Psi}^{0}$.  As usual, you will then use a loop structure to iteratively generate the vectors $\vec{\Psi}^{n}$.  But be careful to create the matrix $\mathbb{T}$ before you enter the loop: this inolves inverting a matrix, which is a time-consuming operation, and since we don't have to do it again at each time step we certaintly do not want to.

#### Plotting Details

<font color = blue>
In order to make it easy to plot the results, write a quick function that takes as inputs the spatial grid parameters $N$ and $x_{\infty}$, and outputs a numpy array of positions.  

## Time Evolution in Free Space

<font color = blue>
Now we want to use our code to analyze the evolution of a wavefunction in free space, where $V(x) = 0$.  We will use two different initial wavefunction shapes:

$$
\psi_A(x) = \frac{1}{\pi^{1/4}\sqrt{\sigma}} \, e^{ip_0x} \, e^{-\frac{(x - x_0)^2}{2\sigma^2}}
$$

and

$$
\psi_B(x) = \left\{\begin{array}{rcl} 0 & \mbox{for} & x < x_0 - \sigma \\ \\ \sqrt{\frac{3}{2\sigma^3}} \, (x - x_0 + \sigma) e^{ip_0x} & \mbox{for} & x_0 - \sigma \le x < x_0 \\ \\ -\sqrt{\frac{3}{2\sigma^3}} \, (x - x_0 - \sigma) e^{ip_0x} & \mbox{for} & x_0 \le x < x_0 + \sigma \\ \\ 0 & \mbox{for} & x_0 + \sigma \le x \end{array}\right.
$$

Both of these wavefunctions are correctly normalized, and are determined by three paramters $\{x_0, p_0, \sigma\}$: the first controls the initial value of $\langle x \rangle$, the second controls the initial value of $\langle p \rangle$, and the third controls the initial uncertainty in both position and momentum.

### Initial Functions

<font color = blue>
Define functions for each of the two initial wavefunction we are interested in.  Plot each, with the choices $x_0 = p_0 = 0$ and $\sigma = 1$.

### Using Time Evolution

#### Visualizing as an Animation

<font color = blue>
    
Starting with the initial wavefunction $\psi_A(x)$ and the parameters $x_0 = p_0 = 0$ and $\sigma = 1$, generate the time evolution of this wavefunction, using grid parameters $x_{\infty} = 20$, $N = 1000$, $T = 5$, and $M = 100$.  Then, create an animation of the *magnitude* of the wavefunction (remember the wavefunction is generally complex, so this step is important).

For creating animations within the jupyter environment, I found the following helpful:

https://www.numfys.net/howto/animations/

https://matplotlib.org/stable/api/animation_api.html

http://louistiao.me/posts/notebooks/embedding-matplotlib-animations-in-jupyter-as-interactive-javascript-widgets/

#### Grid Parameters and Limitations

<font color = blue>
    
Repeat the above exericise, with the same initial wavefunction, but this time with grid parameters $x_{\infty} = 20$, $N = 1000$, $T = 10$, and $M = 100$.  **Discuss** briefly what occurs at the later times, and how grid parameter choices need to be made in order to obtain physically reasonable results.

#### * Exploring the Implications

<font color = blue>
    
Now run four additional examples: two using $\psi_A(x)$ and two using $\psi_B(x)$, for different choices of $\sigma$, $x_0$ and $p_0$.  Make sure you incorporate at least one positive value of $p_0$ and at least one negative value of $p_0$.  In each case, adjust your grid parameter choices to ensure physically reasonable results over a long enouh time frame that you can tell what is happening.  **Discuss** the results thoroughly.

## Time Evolution in the SHO Potential

<font color = blue>
Another classic potential to explore is the SHO potential, which we can non-dimensionalize to be

$$
V(x) = \frac{1}{2}x^2
$$

We want to repeat some of the analysis we just performed for "free space" using this potential.  This time, we will use just the initial Gaussian wavefunction,

$$
\psi_A(x) = \frac{1}{\pi^{1/4}\sqrt{\sigma}} \, e^{ip_0x} \, e^{-\frac{(x - x_0)^2}{2\sigma^2}}
$$

and we will focus on the three cases

$$
x_0 = 0, \ p_0 = 0, \ \sigma = 1
$$

$$
x_0 = 1, \ p_0 = 0, \ \sigma = 1
$$

$$
x_0 = 0, \ p_0 = 1, \ \sigma = 1
$$

In each of these cases you will find the grid parameter choices

$$
x_{\infty} = 10, \ N = 1000, \ T = 20, \ M = 200
$$

to be appropriate.

It will also be helpful for you to know that the first of these cases actually corresponds to an *energy eigenstate* of the quantum SHO system.  (How do states of definite energy evolve over time in quantum mechanics??)

Create animations for each of the three cases, and **discuss** what you see thoroughly.

* Note: you may run into issues with an error message claiming "IOPub data rate exceeded".  If so, you need to edit a config file that sets the allowed limits when you open a Jupyter notebook.

## Final Project Work

<font color = blue>
This week, you should submit your progress on your final project, in a separate Jupyter notebook.  Your notebook should be organized the same way these assignments are (broken into sections, code correctly commented, and short discussions after each section.)  Keep in mind that depending on the topic of your project, I may have very little experience with it, which means your discussions need to be that much better!

At the end, include a brief discussion of what you *intended* to accomplish as compared with what you *did* accomplish, as well as a plan of what you intend to do in the following week.